# Fix INMET files format

In [1]:
import shutil
from unidecode import unidecode
from pathlib import Path
import os


def get_ascii_char(c: str) -> str:
        decoded = unidecode(c)
        if len(decoded) > 1:
            return ''
        ascii_number = ord(decoded)
        ascii_char = bytes([ascii_number]).decode("ascii")
        return ascii_char


def get_ascii_content(content: str) -> str:
    ascii_content = ''
    for c in content:
        ascii_char = get_ascii_char(c)
        ascii_content += ascii_char

    return ascii_content


inmet_folder = os.path.join('data', 'raw_data', 'inmet')
fixed_folder = os.path.join('data', 'raw_data', 'inmet_fixed')


for root, _, files in os.walk(inmet_folder):
    for file in files:
        if not file.lower().endswith('.csv'):
            continue

        file_path = os.path.join(root, file)
        new_file_path = file_path.replace(inmet_folder, fixed_folder)
        Path(new_file_path).parent.mkdir(parents=True, exist_ok=True)

        with open(file_path, 'r', encoding='latin-1') as latin:
            content = latin.read()
            with open(new_file_path, 'w', encoding='utf-8') as fixed:
                fixed.write(get_ascii_content(content))

shutil.rmtree(inmet_folder)
os.rename(fixed_folder, inmet_folder)

# Convert INMET header to columns

In [22]:
import pandas as pd
import os

raw_data_path = os.path.join('data', 'raw_data')
inmet_folder = os.path.join(raw_data_path, 'inmet_fixed')

# Navigate in every csv inside the folder and read it
inmet_df = pd.DataFrame()
for root, _, files in os.walk(inmet_folder):
    for file in files:
        if not file.lower().endswith('.csv'):
            continue

        file_path = os.path.join(root, file)

        meta = pd.read_csv(file_path, nrows=8, header=None)
        metadata = {row[0].strip().replace(':', ''): row[1] for _, row in meta.iterrows()}

        data = pd.read_csv(file_path, skiprows=9, sep=',', decimal=',')
        pass

ParserError: Error tokenizing data. C error: Expected 1 fields in line 5, saw 2
